# 勾配消失問題
多層パーセプトロンでは、層の長さを長くすればするほど表現力は増します。一方で、学習が難しくなるという問題が知られています。

In [ ]:
# PyTorchが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env MKL_NUM_THREADS=1

from torch import set_num_threads, set_num_interop_threads
num_threads = 1
set_num_threads(num_threads)
set_num_interop_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import utils

中間層が10層という深い多層パーセプトロンを用いて、モデル中の重みパラメータの大きさ、勾配の大きさを調べてみます。

In [ ]:
# Modelクラスを継承して新しいクラスを作成します
from torch.nn import Module, Linear, Sigmoid, BCELoss, init


# このModuleではforward()が中間層のノードを出力できるようにカスタマイズしています。
class CustomMLP(Module):
    def __init__(self):
        super().__init__()

        # 中間層が10層の多層パーセプトロン。各レイヤーのノード数は全て50。
        self.linears = []
        self.activations = []
        for i in range(10):
            self.linears += [Linear(in_features=50, out_features=50)]
            self.activations += [Sigmoid()]
        self.linears += [Linear(in_features=50, out_features=1)]
        self.activations += [Sigmoid()]

        # パラメータの初期化
        for layer in self.linears:
            init.normal_(
                layer.weight, mean=0.0, std=1.0
            )  # weight(wij)の初期値。ここでは正規分布に従って初期化する
            init.zeros_(layer.bias)  # bias termの初期値。ここでは0に初期化する。

    def forward(self, inputs, last_node_index=-1):
        x = inputs
        for i, (linear, activation) in enumerate(zip(self.linears, self.activations)):
            x = linear(x)
            x = activation(x)
            if i == last_node_index:
                return x
        else:
            return x


# データセットの生成
nSamples = 1000
nFeatures = 50
# 100個の入力変数を持つイベント1000個生成。それぞれの入力変数は正規分布に従う
x = np.random.randn(nSamples, nFeatures)
# 正解ラベルは0 or 1でランダムに生成
t = np.random.randint(2, size=nSamples).reshape([nSamples, 1])


# numpy -> torch.tensor
from torch import from_numpy

x = from_numpy(x).float()
t = from_numpy(t).float()

# 中間層が10層の多層パーセプトロン。各レイヤーのノード数は全て50。
model = CustomMLP()

# 順伝搬・逆伝搬をして勾配を計算
y_pred = model(x)
loss = BCELoss()(y_pred, t)
loss.backward()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(3, 1, figsize=(6, 12))

# ウェイト(wij)の初期値をプロット
utils.plot_model_weights(model, ax=ax[0])

# 各ノードの出力(sigma(ai))をプロット
utils.plot_model_hidden_nodes(model, x, ax=ax[1])

# ウェイト(wij)の微分(dE/dwij)をプロット
utils.plot_model_weight_gradients(model, x, t, ax=ax[2])

# 図を表示
plt.show()

上段のプロットはパラメータ($w_{ij}$)の初期値を表しています。指定したとおり、各層で正規分布に従って初期化されています。

中段のプロットは活性化関数の出力($z_i$)を表しています。パラメータ($w_{ij}$)の初期値として正規分布を指定すると、シグモイド関数の出力はそのほとんどが0か1に非常に近い値となっています。シグモイド関数の微分は$\sigma^{'}(x)=\sigma(x)\cdot(1-\sigma(x))$なので、$\sigma(x)$が0や1に近いときは微分値も非常に小さな値となります。
誤差逆伝播の式は
$$
\begin{align}
\delta_{i}^{(k)} &= \sigma^{'}(a_i^{(k)}) \left( \sum_j w_{ij}^{(k+1)} \cdot \delta_{j}^{(k+1)} \right) \\
\frac{\partial E_n}{\partial w_{ij}^{(k)}}  &= \delta_{j}^{(k)} \cdot z_{i}^{(k)}
\end{align}
$$
でした。$\sigma^{'}(a_i^{(k)})$が小さいと後方の層から前方の層に誤差が伝わる際に、値が小さくなってしまいます。

下段のプロットは各層での$\frac{\partial E_n}{\partial w_{ij}^{(k)}}$を表しています。
前方の層(0th layer)は後方の層と比較して分布の絶対値が小さくなっています。

このように誤差が前の層にいくにつれて小さくなるため、前の層が後ろの層と比較して学習が進まなくなります。
この問題は勾配消失の問題として知られています。

勾配消失はパラメータの初期値や、活性化関数を変更することによって解決・緩和することがわかっています。
Kerasの
- [初期化のページ](https://keras.io/initializers/)
- [活性化関数のページ](https://keras.io/activations/)

も参考にしながら、この問題の解決を試みてみましょう。

活性化関数・パラメータの初期化方法の変更はそれぞれコード中の"activation"、"initializer"を変更することによって行えます。

例えばパラメータの初期化を(-0.01, +0.01)の一様分布に変更するときは以下のコードのようにすれば良いです。

In [ ]:
from torch.nn import Module, Linear, Sigmoid, BCELoss, init


# このModuleではforward()が中間層のノードを出力できるようにカスタマイズしています。
class CustomMLP(Module):
    def __init__(self):
        super().__init__()

        # 中間層が10層の多層パーセプトロン。各レイヤーのノード数は全て50。
        self.linears = []
        self.activations = []
        for i in range(10):
            self.linears += [Linear(in_features=50, out_features=50)]
            self.activations += [Sigmoid()]
        self.linears += [Linear(in_features=50, out_features=1)]
        self.activations += [Sigmoid()]

        # パラメータの初期化
        for layer in self.linears:
            init.uniform_(layer.weight, a=-0.01, b=+0.01)  # NOTE: 変更箇所
            init.zeros_(layer.bias)  # bias termの初期値。ここでは0に初期化する。

    def forward(self, inputs, last_node_index=-1):
        x = inputs
        for i, (linear, activation) in enumerate(zip(self.linears, self.activations)):
            x = linear(x)
            x = activation(x)
            if i == last_node_index:
                return x
        else:
            return x


# データセットの生成
nSamples = 1000
nFeatures = 50
# 100個の入力変数を持つイベント1000個生成。それぞれの入力変数は正規分布に従う
x = np.random.randn(nSamples, nFeatures)
# 正解ラベルは0 or 1でランダムに生成
t = np.random.randint(2, size=nSamples).reshape([nSamples, 1])


# numpy -> torch.tensor
from torch import from_numpy

x = from_numpy(x).float()
t = from_numpy(t).float()

# 中間層が10層の多層パーセプトロン。各レイヤーのノード数は全て50。
model = CustomMLP()

# 順伝搬・逆伝搬をして勾配を計算
y_pred = model(x)
loss = BCELoss()(y_pred, t)
loss.backward()

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots(3, 1, figsize=(6, 12))

# ウェイト(wij)の初期値をプロット
utils.plot_model_weights(model, ax=ax[0])

# 各ノードの出力(sigma(ai))をプロット
utils.plot_model_hidden_nodes(model, x, ax=ax[1])

# ウェイト(wij)の微分(dE/dwij)をプロット
utils.plot_model_weight_gradients(model, x, t, ax=ax[2])

# 図を表示
plt.show()

この例では活性化関数の出力が0.5付近に集中しています。
どのノードも同じ出力をしているということはノード数を増やした意味があまりなくなっており、多層パーセプトロンの表現力が十分に活かしきれていないことがわかります。
また、勾配消失も先程の例と比較して大きくなっています。